In [1]:
import pandas as pd
import pynmea2
import glob
import os
import numpy as np

In [2]:
files = glob.glob('..\WA2019_V01\LOG\*.log')

In [70]:
def process_time(item):
    gps =pynmea2.parse(item.gps)
    return gps.datetime

def process_file(file):
    data =pd.read_csv(file,delim_whitespace=True,names=['gps'])
    data =data[data.gps.str.startswith('$')]
    data['time'] =data[data.gps.str.startswith('$GPRMC')].apply(process_time,axis=1)
    data.time =data.time.ffill()
    data=data[~data.time.isna()]
    return pd.DataFrame(list(data.apply(process_gps,axis=1)))
    
def process_gps(item):
    try:
        gps =pynmea2.parse(item.gps)
        if gps.sentence_type=='GGA':
            return {'rmc_time':item.time,'sentence':gps.sentence_type,'timestamp':gps.timestamp,'latitude':gps.latitude,'longitude':gps.longitude,'usbl_latitude':np.nan,'usbl_longitude':np.nan,'Heading':np.NaN,'depth_meters':np.NaN}
        if gps.sentence_type=='GLL':
            return {'rmc_time':item.time,'sentence':gps.sentence_type,'timestamp':gps.timestamp,'latitude':np.nan,'longitude':np.nan,'usbl_latitude':gps.latitude,'usbl_longitude':gps.longitude,'Heading':np.NaN,'depth_meters':np.NaN}    
        if gps.sentence_type=='RMC':
            return {'rmc_time':item.time,'sentence':gps.sentence_type,'timestamp':gps.datetime,'latitude':gps.latitude,'longitude':gps.longitude,'usbl_latitude':np.nan,'usbl_longitude':np.nan,'Heading':np.NaN,'depth_meters':np.NaN}
        if gps.sentence_type =='HTD':
            return {'rmc_time':item.time,'sentence':gps.sentence_type,'timestamp':item.time,'latitude':np.nan,'longitude':np.nan,'usbl_latitude':np.nan,'usbl_longitude':np.nan,'Heading':gps.heading,'depth_meters':np.NaN}
        if gps.sentence_type =='DBT':
            return {'rmc_time':item.time,'sentence':gps.sentence_type,'timestamp':item.time,'latitude':np.nan,'longitude':np.nan,'usbl_latitude':np.nan,'usbl_longitude':np.nan,'Heading':np.nan,'depth_meters':gps.depth_meters}
        else: 
            return {'rmc_time':item.time,'sentence':gps.sentence_type,'timestamp':item.time,'latitude':np.nan,'longitude':np.nan,'usbl_latitude':np.nan,'usbl_longitude':np.nan,'Heading':np.nan,'depth_meters':np.NaN}
    except pynmea2.ChecksumError:
        return {'rmc_time':item.time,'sentence':'','timestamp':item.time,'latitude':np.nan,'longitude':np.nan,'usbl_latitude':np.nan,'usbl_longitude':np.nan,'Heading':np.nan,'depth_meters':np.NaN}

In [83]:
for file in files[4:]:
    print(file)
    data =process_file(file)
    data.to_csv(os.path.splitext(file)[0] + '.csv')



..\WA2019_V01\LOG\LOG_WA2019_V01_201912170120191222.log


In [3]:
files

['..\\WA2019_V01\\LOG\\LOG_WA2019_V01_201912170120191218.log',
 '..\\WA2019_V01\\LOG\\LOG_WA2019_V01_201912170120191219.log',
 '..\\WA2019_V01\\LOG\\LOG_WA2019_V01_201912170120191220.log',
 '..\\WA2019_V01\\LOG\\LOG_WA2019_V01_201912170120191221.log',
 '..\\WA2019_V01\\LOG\\LOG_WA2019_V01_201912170120191222.log']

In [93]:
import dask.dataframe as dd
data = dd.read_csv('..\WA2019_V01\LOG\L*.csv')
data['latitude'] = data['latitude'].ffill()
data['longitude']= data['longitude'].ffill()
data['depth_meters'] = data['depth_meters'].ffill()
usbl =data[data.sentence=='GLL'].compute()
usbl.rmc_time =pd.to_datetime(usbl.rmc_time)
usbl.timestamp =pd.to_datetime(usbl.timestamp)


In [94]:
usbl.timestamp =usbl.apply(lambda x: pd.Timestamp.combine(x.rmc_time.date(),x.timestamp.time()), axis=1)



In [95]:
usbl.head()

,Unnamed: 0,Heading,depth_meters,latitude,longitude,rmc_time,sentence,timestamp,usbl_latitude,usbl_longitude
20,20,NaN,75.2,-19.225628,118.826020,2019-12-18 00:00:00,GLL,2019-12-18 00:00:02,-19.225637,118.826010
49,49,NaN,75.2,-19.225637,118.826010,2019-12-18 00:00:02,GLL,2019-12-18 00:00:04,-19.225645,118.825997
78,78,NaN,75.2,-19.225645,118.825997,2019-12-18 00:00:04,GLL,2019-12-18 00:00:06,-19.225653,118.825990
107,107,NaN,75.2,-19.225653,118.825990,2019-12-18 00:00:06,GLL,2019-12-18 00:00:08,-19.225662,118.825982
136,136,NaN,75.0,-19.225662,118.825982,2019-12-18 00:00:08,GLL,2019-12-18 00:00:10,-19.225668,118.825980


In [100]:
ship = usbl[['rmc_time','depth_meters','latitude','longitude']].copy()
ship.set_index('rmc_time',inplace=True)
usbl1 =   usbl[['timestamp','usbl_latitude','usbl_longitude']].copy()
usbl1.set_index('timestamp',inplace=True)
combined =ship.join(usbl1,how='outer')
combined.index.name ='time'
combined.head()

,depth_meters,latitude,longitude,usbl_latitude,usbl_longitude
time,,,,,
2019-12-18 00:00:00,75.2,-19.225628,118.826020,NaN,NaN
2019-12-18 00:00:02,75.2,-19.225637,118.826010,-19.225637,118.826010
2019-12-18 00:00:04,75.2,-19.225645,118.825997,-19.225645,118.825997
2019-12-18 00:00:06,75.2,-19.225653,118.825990,-19.225653,118.825990
2019-12-18 00:00:08,75.0,-19.225662,118.825982,-19.225662,118.825982


In [103]:
combined.to_csv('..\WA2019_V01\LOG\output.csv')